# Foursquare dataset next-POI Recommendation System

First off we import all the necessary libraries:

In [1]:
import polars as rs
import torch
import torch.nn as nn
import torch.nn.functional as F
import lightning as pl
import lightning.pytorch as torchpl
from tqdm import tqdm
import numpy as np
from sklearn.preprocessing import LabelEncoder
from dataclasses import dataclass
import wandb

In [ ]:
import os

# define WANDB_NOTEBOOK_NAME
os.environ["WANDB_NOTEBOOK_NAME"] = "dario_nb"

In [2]:
import gc

gc.collect()
# clean CUDA memory
torch.cuda.empty_cache()

# sometimes jupyter notebook does not release memory, we leave this here so a run-all
# can *sometimes* fix leaks

Next, we load the data, we utilize `polars` since it is much more efficient than `pandas` and can handle large datasets with ease.

In [3]:
columns = ["user", "poi", "date", "TZ"]
data = rs.read_csv(
    "dataset_TIST2015/dataset_TIST2015_Checkins.txt",
    has_header=False,
    low_memory=True,
    separator="\t",
)
data.columns = columns

In [4]:
data

user,poi,date,TZ
i64,str,str,i64
50756,"""4f5e3a72e4b053fd6a4313f6""","""Tue Apr 03 18:00:06 +0000 2012""",240
190571,"""4b4b87b5f964a5204a9f26e3""","""Tue Apr 03 18:00:07 +0000 2012""",180
221021,"""4a85b1b3f964a520eefe1fe3""","""Tue Apr 03 18:00:08 +0000 2012""",-240
66981,"""4b4606f2f964a520751426e3""","""Tue Apr 03 18:00:08 +0000 2012""",-300
21010,"""4c2b4e8a9a559c74832f0de2""","""Tue Apr 03 18:00:09 +0000 2012""",240
…,…,…,…
16349,"""4c957755c8a1bfb7e89024f3""","""Mon Sep 16 23:24:11 +0000 2013""",-240
256757,"""4c8bbb6d9ef0224bd2d6667b""","""Mon Sep 16 23:24:13 +0000 2013""",-180
66425,"""513e82a5e4b0ed4f0f3bcf2d""","""Mon Sep 16 23:24:14 +0000 2013""",-180


Differently from what suggested by the professor, we utilize the full TIST2015 dataset, which has a far greater scale compared to the reduced NY one. However, by following the pruning steps detailed in the paper (http://dx.doi.org/10.1145/3477495.3531989, section 5.1), we obtain sequences that are much smaller in size, resulting in a dataset that is usable on Google Colab's free tier (as required by the assignment).

In [5]:
data_users = (
    data.lazy()
    .group_by("user")
    .agg(
        [
            rs.col("poi").n_unique().alias("n_pois"),
            rs.col("poi").count().alias("n_checkins"),
            # turn the rest into a list
            rs.col("poi").alias("pois"),
            rs.col("date").alias("dates"),
            rs.col("TZ").alias("TZs"),
        ]
    )
).collect()

In [6]:
data_users.describe()

statistic,user,n_pois,n_checkins,pois,dates,TZs
str,f64,f64,f64,f64,f64,f64
"""count""",266909.0,266909.0,266909.0,266909.0,266909.0,266909.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",133455.0,56.477459,124.62537,null,null,null
"""std""",77050.135837,45.968603,140.692138,null,null,null
"""min""",1.0,1.0,1.0,null,null,null
"""25%""",66728.0,30.0,61.0,null,null,null
"""50%""",133455.0,49.0,93.0,null,null,null
"""75%""",200182.0,71.0,148.0,null,null,null
"""max""",266909.0,1246.0,5430.0,null,null,null


## Data Preprocessing

In [7]:
data_culled = data_users.filter(
    (rs.col("n_checkins") > 20) & (rs.col("n_checkins") < 50)
).drop_nulls()

Since the original dataset is huge, we delete it and call the python garbage collector to free up memory. We then proceed with the second pruning step (frequency-based pruning) as detailed in the paper.

In [8]:
del data
del data_users

import gc

gc.collect()

0

In [9]:
# extract unique elements from each lists in data_culled["pois"]
out = data_culled.with_columns(
    [
        rs.col("pois").list.unique(),
        rs.col("pois").list.unique().list.len().alias("n_unique_pois"),
    ]
)

In [10]:
out

user,n_pois,n_checkins,pois,dates,TZs,n_unique_pois
i64,u32,u32,list[str],list[str],list[i64],u32
257050,33,40,"[""4da1b341540ea1cd78c36cde"", ""4bc74b9b8b7c9c748b2836cf"", … ""4f527df9e4b0b589381ab0df""]","[""Sat May 19 18:15:09 +0000 2012"", ""Sat Jun 16 19:55:18 +0000 2012"", … ""Sun Mar 31 23:55:17 +0000 2013""]","[120, 120, … -240]",33
195206,20,24,"[""4f98dec4e4b0eee8ce1f733f"", ""4e60d88062e13e3bcded9e7a"", … ""4ca5c1fb14c33704b09db83b""]","[""Wed Dec 05 01:10:16 +0000 2012"", ""Wed Dec 05 09:22:11 +0000 2012"", … ""Sat Sep 07 13:36:06 +0000 2013""]","[480, 480, … 480]",20
249708,23,27,"[""4e6f565145dd49e0f1bac72a"", ""4b9f0445f964a5202e1037e3"", … ""4c62b770ec94a59302e429ca""]","[""Mon May 28 14:26:24 +0000 2012"", ""Wed May 30 05:14:44 +0000 2012"", … ""Thu Jan 03 03:05:54 +0000 2013""]","[330, 330, … -300]",23
210069,28,30,"[""4b6cf112f964a5203b5f2ce3"", ""4d6b2c0192f6b60c57ed9de0"", … ""4b73785bf964a52022b02de3""]","[""Fri May 11 04:13:48 +0000 2012"", ""Fri May 11 04:26:31 +0000 2012"", … ""Mon Nov 12 07:25:48 +0000 2012""]","[540, 540, … 540]",28
226116,19,43,"[""4b49cb0ff964a520b67326e3"", ""4bfb9c8565fbc9b6f493916c"", … ""4dbd41eb0cb691071cb5d467""]","[""Tue Dec 18 14:56:54 +0000 2012"", ""Wed Jan 02 09:48:05 +0000 2013"", … ""Tue Sep 10 16:10:12 +0000 2013""]","[120, 120, … 180]",19
…,…,…,…,…,…,…
225889,25,27,"[""4ef8f437be7bb4784eea4e29"", ""4bf1c4a052bda5933af2b1b7"", … ""4fd673cae4b0edb666931132""]","[""Thu Jun 07 00:08:28 +0000 2012"", ""Mon Jun 11 18:31:08 +0000 2012"", … ""Fri Jul 06 06:57:51 +0000 2012""]","[-300, -300, … -300]",25
105437,27,38,"[""4fa292a6e4b099a8ddb9e714"", ""4c8250a3dc018cfa5eaecd6c"", … ""4b613d84f964a520d50d2ae3""]","[""Sun Apr 08 12:30:59 +0000 2012"", ""Sun Apr 08 13:56:04 +0000 2012"", … ""Sun Jan 27 07:18:07 +0000 2013""]","[420, 420, … 420]",27
125698,20,23,"[""4ba7ad4cf964a520dca839e3"", ""4bd5c60b9649ce7276a7511d"", … ""4c9cffcb542b224b56d1e59f""]","[""Fri Apr 20 02:23:20 +0000 2012"", ""Sat May 19 21:05:02 +0000 2012"", … ""Sun Aug 12 17:08:22 +0000 2012""]","[-240, -240, … -300]",20


In [11]:
l = out["pois"][0].to_list()
len(set(l))  # print number of unique POIs in first sequence

33

In [12]:
l2 = data_culled["pois"][0].to_list()
len(l2)  # print sequence length of first user

40

In [13]:
len(set(l2))  # confirm that the two match

33

In [14]:
# run a Polars query to obtain all the frequent POIs, the ones expected to survive the filtering
unique_pois = out["pois"]
frequent_pois = unique_pois.list.explode().value_counts().filter(rs.col("count") >= 10)

In [15]:
frequent_pois

pois,count
str,u32
"""4e5e3644814d9e0233e0151e""",16
"""4b2ae76ff964a520d6b124e3""",11
"""4b52966cf964a520328327e3""",18
"""4c25a376a852c9283e54e56c""",17
"""4c30658a16adc928711bbf9c""",44
…,…
"""4a65d6a4f964a520a1c71fe3""",100
"""4b67e81bf964a520bb602be3""",78
"""4bae2c85f964a520a98d3be3""",17


In [16]:
frequent_pois = frequent_pois["pois"]
frequent_pois = set(frequent_pois.to_list())

In [17]:
data_culled

user,n_pois,n_checkins,pois,dates,TZs
i64,u32,u32,list[str],list[str],list[i64]
257050,33,40,"[""4da1a9e0c6e96ea8a6e2c45d"", ""4c473713417b20a160d5dca9"", … ""4b3d350ff964a520469025e3""]","[""Sat May 19 18:15:09 +0000 2012"", ""Sat Jun 16 19:55:18 +0000 2012"", … ""Sun Mar 31 23:55:17 +0000 2013""]","[120, 120, … -240]"
195206,20,24,"[""4e60d88062e13e3bcded9e7a"", ""503dfc8ce4b0a5c85898cfd9"", … ""50cb5f14e4b029bc9c97f4fc""]","[""Wed Dec 05 01:10:16 +0000 2012"", ""Wed Dec 05 09:22:11 +0000 2012"", … ""Sat Sep 07 13:36:06 +0000 2013""]","[480, 480, … 480]"
249708,23,27,"[""4f4dd4e1e4b0393387511980"", ""4b55d511f964a520fef127e3"", … ""4d58d7fc7e22370425e6a973""]","[""Mon May 28 14:26:24 +0000 2012"", ""Wed May 30 05:14:44 +0000 2012"", … ""Thu Jan 03 03:05:54 +0000 2013""]","[330, 330, … -300]"
210069,28,30,"[""4b52c581f964a520c28727e3"", ""4b5ea776f964a520119529e3"", … ""4eedf73ee3002155941114e8""]","[""Fri May 11 04:13:48 +0000 2012"", ""Fri May 11 04:26:31 +0000 2012"", … ""Mon Nov 12 07:25:48 +0000 2012""]","[540, 540, … 540]"
226116,19,43,"[""4d0783af0e49721ecb8a807f"", ""4d0783af0e49721ecb8a807f"", … ""4d412a994acd236a1360f7a9""]","[""Tue Dec 18 14:56:54 +0000 2012"", ""Wed Jan 02 09:48:05 +0000 2013"", … ""Tue Sep 10 16:10:12 +0000 2013""]","[120, 120, … 180]"
…,…,…,…,…,…
225889,25,27,"[""4b936956f964a5200e4334e3"", ""4a36fbcbf964a5201b9e1fe3"", … ""4f4016a2e4b0f7b458b21252""]","[""Thu Jun 07 00:08:28 +0000 2012"", ""Mon Jun 11 18:31:08 +0000 2012"", … ""Fri Jul 06 06:57:51 +0000 2012""]","[-300, -300, … -300]"
105437,27,38,"[""4baa0e9cf964a52034473ae3"", ""4b9a639df964a52017b135e3"", … ""4b4479a4f964a5205ef525e3""]","[""Sun Apr 08 12:30:59 +0000 2012"", ""Sun Apr 08 13:56:04 +0000 2012"", … ""Sun Jan 27 07:18:07 +0000 2013""]","[420, 420, … 420]"
125698,20,23,"[""4cb0dc48c5e6a1cd2c10d9f6"", ""3fd66200f964a52000ee1ee3"", … ""4b81c860f964a520c6bd30e3""]","[""Fri Apr 20 02:23:20 +0000 2012"", ""Sat May 19 21:05:02 +0000 2012"", … ""Sun Aug 12 17:08:22 +0000 2012""]","[-240, -240, … -300]"


In [18]:
data_culled = data_culled.with_columns(
    [
        rs.col("pois")
        .list.eval(
            rs.element().is_in(frequent_pois),
        )
        .alias("is_frequent")
    ]
)  # prep mask

In [19]:
final_data = (
    data_culled.lazy()
    .with_row_index()
    .explode(
        [
            "pois",
            "dates",
            "TZs",
            "is_frequent",
        ]
    )
    .group_by("user")
    .agg(
        [
            rs.col("pois").filter(rs.col("is_frequent")).alias("pois"),
            rs.col("dates").filter(rs.col("is_frequent")).alias("dates"),
            rs.col("TZs").filter(rs.col("is_frequent")).alias("TZs"),
            rs.col("pois").filter(rs.col("is_frequent")).n_unique().alias("n_pois"),
            rs.col("pois").filter(rs.col("is_frequent")).count().alias("n_checkins"),
        ]
    )
    .filter(rs.col("n_checkins") > 0)
    .filter(rs.col("n_pois") > 0)
    .collect()
)  # filter out infrequent pois and users with no pois

In [20]:
final_data.describe()

statistic,user,pois,dates,TZs,n_pois,n_checkins
str,f64,f64,f64,f64,f64,f64
"""count""",19862.0,19862.0,19862.0,19862.0,19862.0,19862.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",156852.822274,null,null,null,6.123452,8.831437
"""std""",76314.892884,null,null,null,4.609024,6.877662
"""min""",49.0,null,null,null,1.0,1.0
"""25%""",95613.0,null,null,null,3.0,4.0
"""50%""",167846.0,null,null,null,5.0,7.0
"""75%""",224576.0,null,null,null,8.0,12.0
"""max""",266909.0,null,null,null,32.0,46.0


At this stage, culling is done, we can appreciate that `polars`'s SQL/functional-style API is different from Pandas, but it is very powerful and efficient.

The next step is geohashing the POIs, that is, we want to convert the latitude-longitude positions of the POIs into a grid-based geohash representation, which will form the basis for our network's embeddings.

In [21]:
import geohash2 as gh

pois = rs.read_csv(
    "dataset_TIST2015/dataset_TIST2015_POIs.txt",
    has_header=False,
    low_memory=True,
    separator="\t",
)
pois.columns = ["poi", "lat", "long", "category", "country"]
pois = pois.drop("category").drop("country")

In [22]:
pois = (
    pois.lazy()
    .filter(rs.col("poi").is_in(frequent_pois))
    .select(
        [
            rs.col("poi"),
            rs.struct(
                [
                    rs.col("lat").cast(rs.Float32),
                    rs.col("long").cast(rs.Float32),
                ]
            )
            .alias("location")
            .map_elements(
                lambda s: gh.encode(s["lat"], s["long"], precision=6),
                return_dtype=rs.String,
            )
            .alias("geohash"),
        ]
    )
    .collect()
)
poi_geo_dict = dict(zip(pois["poi"], pois["geohash"]))

In [23]:
# for each row in final_data, add the geohash of the pois by hitting the poi_geo_dict

final_data = final_data.with_columns(
    [
        rs.col("pois")
        .map_elements(
            lambda s: [poi_geo_dict[s] for s in s],
            return_dtype=rs.List(rs.String),
        )
        .alias("geohashes")
    ]
)

In [24]:
final_data["dates"][79].to_list()  # check out a temporal sequence

['Sat Sep 15 18:08:37 +0000 2012',
 'Thu Jan 03 00:46:51 +0000 2013',
 'Tue Apr 02 01:22:47 +0000 2013',
 'Sat Apr 06 02:22:27 +0000 2013',
 'Sun Aug 04 23:48:22 +0000 2013']

In [25]:
final_data["TZs"][79].to_list()  # ... and the corresponding timezones

[-300, -360, -360, -360, -300]

The work *might* seem over, however, we still have timezones to account for, we want to normalize everything according to GMT, so we convert the timestamps accordingly.

In [26]:
import datetime


def UTC_to_local(utc, tz):

    date = datetime.datetime.strptime(utc, "%a %b %d %H:%M:%S %z %Y")
    date = date.replace(tzinfo=datetime.timezone.utc)

    # shift by tz offset
    date = date.astimezone(datetime.timezone(datetime.timedelta(minutes=tz)))

    date_s = datetime.datetime.strftime(date, "%Y-%m-%d %H:%M:%S")
    return date_s


def to_UNIX_time(date):
    return datetime.datetime.strptime(
        date, "%Y-%m-%d %H:%M:%S"
    ).timestamp()  # we use UNIX time as a key for sorting the POIs in our polars query

In [27]:
UTC_to_local("Mon May 21 15:53:01 +0000 2012", -420)  # example of usage

'2012-05-21 08:53:01'

In [28]:
final_data = final_data.with_columns(
    [
        rs.struct([rs.col("dates"), rs.col("TZs")])
        .alias("times")
        .map_elements(
            lambda struct: [
                UTC_to_local(date, tz)
                for date, tz in zip(struct["dates"], struct["TZs"])
            ],
            return_dtype=rs.List(rs.String),
        )
    ]
)  # This performs timezone conversion

In [29]:
final_sorted = final_data.select(  # sort the times
    [
        rs.col("user"),
        rs.struct(
            [
                rs.col("pois"),
                rs.col("times"),
            ]
        ).map_elements(
            lambda struct: [
                poi
                for poi, _ in sorted(
                    zip(  # here we sort the POIs struct by UNIX timestamps of the GMT times
                        struct["pois"], [to_UNIX_time(date) for date in struct["times"]]
                    ),
                    key=lambda s: s[1],
                )
            ],
            return_dtype=rs.List(rs.String),
        ),
        rs.struct(
            [
                rs.col("geohashes"),
                rs.col("times"),
            ]
        ).map_elements(
            lambda struct: [
                geo
                for geo, _ in sorted(
                    zip(
                        struct["geohashes"],  # same thing goes on for geohashes
                        [to_UNIX_time(date) for date in struct["times"]],
                    ),
                    key=lambda s: s[1],
                )
            ],
            return_dtype=rs.List(rs.String),
        ),
        rs.col("times")
        .map_elements(
            lambda dates: sorted(dates, key=to_UNIX_time),
            return_dtype=rs.List(rs.String),
        )
        .alias("times_sorted"),
        rs.col("n_checkins"),
    ]
)

# P.S, admittedly, it would have been more efficient to encode the geohashes *after* sorting the POIs,
# so that we could save on the sorting of the geohashes. Tough luck, you can't win 'em all.

In [30]:
final_sorted

user,pois,geohashes,times_sorted,n_checkins
i64,list[str],list[str],list[str],u32
224681,"[""49c2fff9f964a5204d561fe3"", ""44d17cecf964a5202b361fe3"", … ""4a469855f964a5202aa91fe3""]","[""dqcjpr"", ""dqcjnb"", … ""dqcjqc""]","[""2012-04-16 16:10:05"", ""2012-07-03 09:44:34"", … ""2013-06-23 11:13:06""]",8
146176,"[""4b924f49f964a5206af133e3"", ""4bac260bf964a5204de73ae3"", … ""4bd7df7d5cf276b0342c9c00""]","[""w281w9"", ""w30dy9"", … ""w28678""]","[""2012-10-12 19:44:55"", ""2013-01-02 21:25:30"", … ""2013-01-26 17:15:53""]",6
59972,"[""4e5a143618388cd5cbac0130"", ""4ca1c5fdd3c2b60cd31de8bc"", … ""50c9e22ae4b0fe058fc7d378""]","[""qqguf7"", ""qqgu6z"", … ""qqgudg""]","[""2012-05-27 19:12:54"", ""2012-06-11 13:35:26"", … ""2013-08-26 14:30:04""]",16
261037,"[""4b5823adf964a520fa4b28e3"", ""4b6180a5f964a520e2152ae3"", … ""4e5c49741850ca78f0812408""]","[""xn0m80"", ""xn0me7"", … ""xn76st""]","[""2012-06-10 12:12:28"", ""2012-06-10 19:09:18"", … ""2013-04-30 15:49:59""]",5
154367,"[""474d8bbdf964a520a54c1fe3"", ""4aa76d20f964a520d24c20e3"", … ""411c0480f964a5201a0c1fe3""]","[""drt2zj"", ""dhvr1v"", … ""dr46zg""]","[""2012-09-09 19:34:25"", ""2012-10-24 19:11:09"", … ""2013-05-19 08:47:05""]",34
…,…,…,…,…
259399,"[""4b53e2b4f964a520f9ad27e3"", ""4bf269b79f93952137b03098"", … ""42323100f964a5200b201fe3""]","[""qqu8b8"", ""qqgvp2"", … ""9v6kpv""]","[""2012-05-12 16:10:31"", ""2012-06-21 19:53:29"", … ""2013-03-11 03:29:10""]",11
89845,"[""4b565926f964a5209b0c28e3"", ""4b565926f964a5209b0c28e3"", … ""4b2af373f964a520beb224e3""]","[""xn77fd"", ""xn77fd"", … ""xn76um""]","[""2012-11-26 12:11:36"", ""2012-12-10 08:58:40"", … ""2013-08-14 16:17:07""]",5
89693,"[""41059b00f964a520850b1fe3"", ""4541114bf964a520433c1fe3"", … ""4541114bf964a520433c1fe3""]","[""9q8vzp"", ""9mudjn"", … ""9mudjn""]","[""2012-06-03 16:59:24"", ""2012-06-03 19:53:48"", … ""2013-01-25 10:51:40""]",4


we now need to obtain a dataframe containing: each POI, it's geohash, and a set of all the check-ins it appears in
this is just one `polars` query away!

In [31]:
pois_checkins = final_sorted.explode(["pois", "geohashes"]).drop("n_checkins")

pois_checkins = (
    pois_checkins.with_columns(
        [
            rs.col("geohashes").map_elements(lambda s: s[:4], rs.String).alias("g4"),
        ]
    )
    .drop("geohashes")
    .group_by(["pois", "g4"])
    .agg([rs.col("times_sorted").flatten().alias("checkin_times")])
)

In [32]:
pois_checkins  # with this we can *efficiently* build our POI-POI spatial-temporal graphs

pois,g4,checkin_times
str,str,list[str]
"""4b7fd336f964a520733f30e3""","""sxk9""","[""2012-10-16 12:46:22"", ""2012-10-16 12:54:51"", … ""2013-08-27 17:23:14""]"
"""4e58e6a9d164da2b275a8135""","""syg4""","[""2012-10-22 14:56:32"", ""2012-10-23 19:11:11"", … ""2013-05-23 18:31:44""]"
"""4ae9e965f964a5208db721e3""","""dnru""","[""2012-04-15 16:54:57"", ""2012-04-22 15:52:18"", … ""2013-08-06 11:47:25""]"
"""4d15ec39816af04d325846c2""","""sxk9""","[""2012-12-01 19:42:45"", ""2012-12-01 19:43:49"", … ""2013-08-27 12:53:39""]"
"""4bab4206f964a520129c3ae3""","""srbj""","[""2012-05-12 15:08:26"", ""2012-05-13 19:16:40"", … ""2012-06-08 15:11:22""]"
…,…,…
"""4f3a6a34a17c224158ecd541""","""u173""","[""2012-08-05 17:09:50"", ""2012-09-05 20:32:36"", … ""2013-03-17 16:08:43""]"
"""4b5a79a1f964a52058c728e3""","""xn77""","[""2012-05-07 18:12:17"", ""2012-05-08 19:53:36"", … ""2013-07-24 16:25:03""]"
"""4ad4bffef964a520f5ea20e3""","""9v1z""","[""2012-06-08 10:09:30"", ""2012-06-09 11:44:10"", … ""2012-05-19 11:04:02""]"


In [33]:
def UTC_to_weekslot(utc: str) -> int:
    """UTC_to_weekslot converts a UTC timestamp to a weekslot.

    Parameters
    ----------
    utc : str
        A string representing a UTC timestamp.

    Returns
    -------
    int
        A weekslot in the range [0, 56).
    """

    date = datetime.datetime.strptime(utc, "%Y-%m-%d %H:%M:%S")
    week = date.weekday()
    hour = date.hour

    return week * 8 + hour // 3

Next, we want to encode all of our inputs for our neural networks, this could *probably* be done 
with polars magic, but it's too delicate and we prefer classic for-looping.

In [34]:
encoder_dict = {
    "users": LabelEncoder(),
    "pois": LabelEncoder(),
    "g2": LabelEncoder(),
    "g3": LabelEncoder(),
    "g4": LabelEncoder(),
    "g5": LabelEncoder(),
    "g6": LabelEncoder(),
}

encoded_data = {
    "users": [],
    "pois": [],
    "g2": [],
    "g3": [],
    "g4": [],
    "g5": [],
    "g6": [],
}

unique_data = {
    "users": set(),
    "pois": set(),
    "g2": set(),
    "g3": set(),
    "g4": set(),
    "g5": set(),
    "g6": set(),
}

# quick and dirty encoding:
# 1. put every unique symbol in a list
# 2. fit the respective encoder
# 3. transform the lists

for i, row in enumerate(final_sorted.iter_rows()):

    user, pois, geohashes, times_sorted, n_checkins = row

    g2 = [geo[:2] for geo in geohashes]
    g3 = [geo[:3] for geo in geohashes]
    g4 = [geo[:4] for geo in geohashes]
    g5 = [geo[:5] for geo in geohashes]
    g6 = [geo[:6] for geo in geohashes]  # redundant, but I like symmetry

    unique_data["users"].add(user)
    unique_data["pois"].update(pois)
    unique_data["g2"].update(g2)
    unique_data["g3"].update(g3)
    unique_data["g4"].update(g4)
    unique_data["g5"].update(g5)
    unique_data["g6"].update(g6)

for property, enc, data in zip(
    encoder_dict.keys(), encoder_dict.values(), unique_data.values()
):
    enc.fit(list(data))
    encoder_dict[property] = enc

In [35]:
# this could be optimized, right now it takes a while, at least we have a nice progress bar to look at

ds_size = len(final_sorted)

for i, row in tqdm(enumerate(final_sorted.iter_rows()), total=ds_size):

    user, pois, geohashes, times_sorted, n_checkins = row

    g2 = [geo[:2] for geo in geohashes]
    g3 = [geo[:3] for geo in geohashes]
    g4 = [geo[:4] for geo in geohashes]
    g5 = [geo[:5] for geo in geohashes]
    g6 = [geo[:6] for geo in geohashes]

    encoded_data["users"].append(encoder_dict["users"].transform([user])[0])
    encoded_data["pois"].append(encoder_dict["pois"].transform(pois))
    encoded_data["g2"].append(encoder_dict["g2"].transform(g2))
    encoded_data["g3"].append(encoder_dict["g3"].transform(g3))
    encoded_data["g4"].append(encoder_dict["g4"].transform(g4))
    encoded_data["g5"].append(encoder_dict["g5"].transform(g5))
    encoded_data["g6"].append(encoder_dict["g6"].transform(g6))

    # sum 1 to all values to avoid 0s
    encoded_data["users"][-1] += 1
    encoded_data["pois"][-1] += 1
    encoded_data["g2"][-1] += 1
    encoded_data["g3"][-1] += 1
    encoded_data["g4"][-1] += 1
    encoded_data["g5"][-1] += 1
    encoded_data["g6"][-1] += 1

100%|██████████| 19862/19862 [02:56<00:00, 112.76it/s]


In [36]:
# check that we left space for the padding token
min((arr.min() for arr in encoded_data["pois"]))

1

In [37]:
pois_checkins

pois,g4,checkin_times
str,str,list[str]
"""4b7fd336f964a520733f30e3""","""sxk9""","[""2012-10-16 12:46:22"", ""2012-10-16 12:54:51"", … ""2013-08-27 17:23:14""]"
"""4e58e6a9d164da2b275a8135""","""syg4""","[""2012-10-22 14:56:32"", ""2012-10-23 19:11:11"", … ""2013-05-23 18:31:44""]"
"""4ae9e965f964a5208db721e3""","""dnru""","[""2012-04-15 16:54:57"", ""2012-04-22 15:52:18"", … ""2013-08-06 11:47:25""]"
"""4d15ec39816af04d325846c2""","""sxk9""","[""2012-12-01 19:42:45"", ""2012-12-01 19:43:49"", … ""2013-08-27 12:53:39""]"
"""4bab4206f964a520129c3ae3""","""srbj""","[""2012-05-12 15:08:26"", ""2012-05-13 19:16:40"", … ""2012-06-08 15:11:22""]"
…,…,…
"""4f3a6a34a17c224158ecd541""","""u173""","[""2012-08-05 17:09:50"", ""2012-09-05 20:32:36"", … ""2013-03-17 16:08:43""]"
"""4b5a79a1f964a52058c728e3""","""xn77""","[""2012-05-07 18:12:17"", ""2012-05-08 19:53:36"", … ""2013-07-24 16:25:03""]"
"""4ad4bffef964a520f5ea20e3""","""9v1z""","[""2012-06-08 10:09:30"", ""2012-06-09 11:44:10"", … ""2012-05-19 11:04:02""]"


In [38]:
# we also encode the graph dataframe so we can build the graphs

pois_checkins = (
    pois_checkins.lazy()
    .with_columns(
        [
            rs.col("pois").map_elements(
                lambda s: encoder_dict["pois"].transform([s])[0] + 1, rs.Int64
            ),
            rs.col("g4").map_elements(
                lambda s: encoder_dict["g4"].transform([s])[0] + 1, rs.Int64
            ),  # apply utc_to_weekslot to each timestamp in the list
            rs.col("checkin_times").map_elements(
                lambda s: [UTC_to_weekslot(date) for date in s], rs.List(rs.Int64)
            ),
        ]
    )
    .sort("pois")
    .collect()
)

In [39]:
# add fictitious POI 0 to the graph, with nonexistent geohash and no timeslot, so we get a 0 row and column for the padding token
fake_datapoint = rs.DataFrame(
    {
        "pois": [0],
        "g4": [pois_checkins["g4"].max() + 42],
        "checkin_times": [[43]],
    }
)
# this is a lot of work since polars dataframes are immutable by default, we have to run a query to change the 43 into an empty list
# we NEED the 43 otherwise polars won't infer the datatype of the list

fake_datapoint = fake_datapoint.with_columns(
    [rs.col("checkin_times").map_elements(lambda s: [], rs.List(rs.Int64))]
)

pois_checkins = fake_datapoint.vstack(pois_checkins)

In [40]:
spatial_row = np.array(pois_checkins["g4"].to_list()).reshape(-1, 1)

In [41]:
# outer product using equality
spatial_graph = (spatial_row == spatial_row.T).astype(np.int32)
spatial_graph[0, 0] = (
    0  # the fake g4 is still equal to itself, we suppress this equality
)

In [42]:
temporal_row = pois_checkins["checkin_times"].to_list()

In [43]:
temporal_graph = np.zeros((spatial_row.shape[0], spatial_row.shape[0]))

In [44]:
temporal_sets = [np.array(list(set(row))) for row in temporal_row]

In [45]:
time_sets = torch.zeros((len(temporal_sets), 56), dtype=torch.int8)

for i, r in enumerate(temporal_row):
    indices = torch.tensor(r, dtype=torch.long)
    time_sets[i, indices] = 1

In [46]:
time_sets.shape

torch.Size([4456, 56])

In [47]:
# AND outer product

intersection = time_sets @ time_sets.T
union = time_sets.unsqueeze(1) | time_sets.unsqueeze(0)
union = union.sum(dim=2)
iou = intersection / union

In [48]:
temporal_graph = iou >= 0.9
# cast to int
temporal_graph = temporal_graph.int()

In [49]:
temporal_graph[0, :].sum()

tensor(0)

We print information about the sparsity of the graphs, we note that 
the sparsity of the graphs is similar to that of the paper.

In [50]:
temporal_density = (
    temporal_graph.sum() / (temporal_graph.shape[0] * temporal_graph.shape[1])
).item()
spatial_density = (
    spatial_graph.sum() / (spatial_graph.shape[0] * spatial_graph.shape[1])
).item()

print(f"Temporal sparsity: {(1 - temporal_density) * 100:.2f}%")

print(f"Spatial sparsity: {(1 - spatial_density) * 100:.2f}%")

Temporal sparsity: 97.05%
Spatial sparsity: 96.99%


## Dataset and Datamodule

We then define a pytorch dataset and a custom collation function that allows us to dynamically
pad sequences to the longest one in the batch (as opposed to the longest one in the dataset)
as they are loaded during training, this gives us an edge in performance by dramatically reducing the 
sparsity of our inputs.

In [51]:
from torch.utils.data import Dataset
from torch.nn.utils.rnn import pad_sequence


def rnn_collation_fn(batch):

    users = []
    pois = []
    g2 = []
    g3 = []
    g4 = []
    g5 = []
    g6 = []

    for user, poi, geo2, geo3, geo4, geo5, geo6 in batch:
        users.append(user)
        pois.append(poi)
        g2.append(geo2)
        g3.append(geo3)
        g4.append(geo4)
        g5.append(geo5)
        g6.append(geo6)
    seq = (
        torch.tensor(users, dtype=torch.long),
        pad_sequence(pois, batch_first=True, padding_value=0),
        pad_sequence(g2, batch_first=True, padding_value=0),
        pad_sequence(g3, batch_first=True, padding_value=0),
        pad_sequence(g4, batch_first=True, padding_value=0),
        pad_sequence(g5, batch_first=True, padding_value=0),
        pad_sequence(g6, batch_first=True, padding_value=0),
    )  # build a sequence

    x = (
        seq[0],
        seq[1][:, :-1],
        seq[2][:, :-1],
        seq[3][:, :-1],
        seq[4][:, :-1],
        seq[5][:, :-1],
        seq[6][:, :-1],
    )  # omit the last one for sample

    y = (
        seq[0],
        seq[1][:, 1:],
        seq[2][:, 1:],
        seq[3][:, 1:],
        seq[4][:, 1:],
        seq[5][:, 1:],
        seq[6][:, 1:],
    )  # omit the first one for ground truth

    return x, y


class CheckinDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data["users"])

    def __getitem__(self, idx):

        x = (
            torch.tensor(encoded_data["users"][idx], dtype=torch.long),
            torch.tensor(encoded_data["pois"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g2"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g3"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g4"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g5"][idx], dtype=torch.long),
            torch.tensor(encoded_data["g6"][idx], dtype=torch.long),
        )

        return x

In [52]:
class CheckinModule(pl.LightningDataModule):
    def __init__(self, encoded_data, batch_size=32, workers=4):
        super().__init__()
        self.encoded_data = encoded_data
        self.batch_size = batch_size
        self.workers = workers

    def setup(self, stage=None):
        self.whole_dataset = CheckinDataset(self.encoded_data)

        l = len(self.whole_dataset)

        train_size = int(0.8 * l)
        val_size = int(0.1 * l)
        test_size = l - train_size - val_size

        # generate train, val, test datasets by random split
        self.train_dataset, self.val_dataset, self.test_dataset = (
            torch.utils.data.random_split(
                self.whole_dataset, [train_size, val_size, test_size]
            )
        )

    def train_dataloader(self):
        return torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=self.batch_size,
            shuffle=True,
            num_workers=self.workers,
            collate_fn=rnn_collation_fn,
        )

    def val_dataloader(self):
        return torch.utils.data.DataLoader(
            self.val_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.workers,
            collate_fn=rnn_collation_fn,
        )

    def test_dataloader(self):
        return torch.utils.data.DataLoader(
            self.test_dataset,
            batch_size=self.batch_size,
            shuffle=False,
            num_workers=self.workers,
            collate_fn=rnn_collation_fn,
        )

    def save(self, whole_path, train_path, val_path, test_path):
        torch.save(self.whole_dataset, whole_path)
        torch.save(self.train_dataset, train_path)
        torch.save(self.val_dataset, val_path)
        torch.save(self.test_dataset, test_path)

    @staticmethod  # load without instantiating
    def load(whole_path, train_path, val_path, test_path):
        whole_dataset = torch.load(whole_path)
        train_dataset = torch.load(train_path)
        val_dataset = torch.load(val_path)
        test_dataset = torch.load(test_path)
        return whole_dataset, train_dataset, val_dataset, test_dataset

## Baseline model: LSTM

In [53]:
@dataclass
class BaselineDimensions:
    nuser: int
    npoi: int
    g2len: int
    g3len: int
    g4len: int
    g5len: int
    g6len: int


class HMT_RN(pl.LightningModule):
    def __init__(
        self,
        dimensions: BaselineDimensions,
        embedding_dim,
        lstm_hidden_dim,
        dropout_rate=0.2,  # 0.9 is a lot, but the paper says so.
    ):
        super(HMT_RN, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = lstm_hidden_dim
        self.dims = dimensions

        # Embedding layers one for user, one for poi and one for each G@P
        self.user_embedding = nn.Embedding(
            dimensions.nuser, embedding_dim, padding_idx=0
        )
        self.poi_embedding = nn.Embedding(dimensions.npoi, embedding_dim, padding_idx=0)
        self.g2_embed = nn.Embedding(dimensions.g2len, embedding_dim, padding_idx=0)
        self.g3_embed = nn.Embedding(dimensions.g3len, embedding_dim, padding_idx=0)
        self.g4_embed = nn.Embedding(dimensions.g4len, embedding_dim, padding_idx=0)
        self.g5_embed = nn.Embedding(dimensions.g5len, embedding_dim, padding_idx=0)
        self.g6_embed = nn.Embedding(dimensions.g6len, embedding_dim, padding_idx=0)

        # Dropout layer for embeddings
        self.e_drop = nn.Dropout(p=dropout_rate)

        # LSTM layer
        self.lstm = nn.LSTM(
            input_size=embedding_dim, hidden_size=lstm_hidden_dim, batch_first=True
        )

        # Linear layers for prediction tasks
        self.linear_poi = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.npoi)
        self.linear_g2 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g2len)
        self.linear_g3 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g3len)
        self.linear_g4 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g4len)
        self.linear_g5 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g5len)
        self.linear_g6 = nn.Linear(lstm_hidden_dim + embedding_dim, dimensions.g6len)

        self.criterion = nn.CrossEntropyLoss()
        self.apply(self.init_weights)

    def init_weights(self, w):

        if type(w) == nn.Linear:
            nn.init.kaiming_normal_(w.weight)
            nn.init.constant_(w.bias, 0)
        elif type(w) == nn.LSTM:
            for name, param in w.named_parameters():
                if "bias" in name:
                    nn.init.constant_(param, 0)
                elif "weight" in name:
                    nn.init.kaiming_normal_(param)
        elif type(w) == nn.Embedding:
            nn.init.kaiming_normal_(w.weight)
            nn.init.constant_(w.weight[0], 0)

    def forward(self, batch):
        """forward passes the batch through the model.

        Parameters
        ----------
        batch : `tuple[torch.Tensor]`
            A tuple of tensors ordered as follows:
            (users, poi, x_geoHash2, x_geoHash3, x_geoHash4, x_geoHash5, x_geoHash6)
        """

        users, poi, x_geoHash2, x_geoHash3, x_geoHash4, x_geoHash5, x_geoHash6 = batch

        B, T = poi.shape

        # make it so  that users are tiled T times
        users = users.repeat(T, 1).T

        e_user = self.e_drop(self.user_embedding(users))
        e_poi = self.e_drop(self.poi_embedding(poi))
        e_gap2 = self.e_drop(self.g2_embed(x_geoHash2))
        e_gap3 = self.e_drop(self.g3_embed(x_geoHash3))
        e_gap4 = self.e_drop(self.g4_embed(x_geoHash4))
        e_gap5 = self.e_drop(self.g5_embed(x_geoHash5))
        e_gap6 = self.e_drop(self.g6_embed(x_geoHash6))

        h_t, c_t = self.lstm(e_poi)

        # dense layers
        next_poi = self.linear_poi(torch.cat((h_t, e_poi), dim=2))
        next_g2 = self.linear_g2(torch.cat((h_t, e_gap2), dim=2))
        next_g3 = self.linear_g3(torch.cat((h_t, e_gap3), dim=2))
        next_g4 = self.linear_g4(torch.cat((h_t, e_gap4), dim=2))
        next_g5 = self.linear_g5(torch.cat((h_t, e_gap5), dim=2))
        next_g6 = self.linear_g6(torch.cat((h_t, e_gap6), dim=2))

        return next_poi, next_g2, next_g3, next_g4, next_g5, next_g6

    def training_step(self, batch, batch_idx):
        x, y = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6
        self.log("train/loss", loss)
        self.log("train/loss_gap2", loss_gap2)
        self.log("train/loss_gap3", loss_gap3)
        self.log("train/loss_gap4", loss_gap4)
        self.log("train/loss_gap5", loss_gap5)
        self.log("train/loss_gap6", loss_gap6)
        self.log("train/loss_poi", loss_poi)

        return {"loss": loss}

    def validation_step(self, batch, batch_idx):
        x, y = batch
        (
            poi_pred,
            gap2_pred,
            gap3_pred,
            gap4_pred,
            gap5_pred,
            gap6_pred,
        ) = self(x)

        loss_poi = self.criterion(
            poi_pred.reshape(-1, self.dims.npoi), y[1].reshape(-1)
        )
        loss_gap2 = self.criterion(
            gap2_pred.reshape(-1, self.dims.g2len), y[2].reshape(-1)
        )
        loss_gap3 = self.criterion(
            gap3_pred.reshape(-1, self.dims.g3len), y[3].reshape(-1)
        )
        loss_gap4 = self.criterion(
            gap4_pred.reshape(-1, self.dims.g4len), y[4].reshape(-1)
        )
        loss_gap5 = self.criterion(
            gap5_pred.reshape(-1, self.dims.g5len), y[5].reshape(-1)
        )
        loss_gap6 = self.criterion(
            gap6_pred.reshape(-1, self.dims.g6len), y[6].reshape(-1)
        )

        loss = (
            loss_poi + loss_gap2 + loss_gap3 + loss_gap4 + loss_gap5 + loss_gap6
        ) / 6

        self.log("val/loss", loss)
        self.log("val/loss_gap2", loss_gap2)
        self.log("val/loss_gap3", loss_gap3)
        self.log("val/loss_gap4", loss_gap4)
        self.log("val/loss_gap5", loss_gap5)
        self.log("val/loss_gap6", loss_gap6)
        self.log("val/loss_poi", loss_poi)

        return loss

    def configure_optimizers(self):
        # Define optimizer and scheduler
        optimizer = torch.optim.AdamW(self.parameters(), lr=1e-4, amsgrad=True)
        return optimizer

In [54]:
n_users = encoder_dict["users"].classes_.shape[0]
n_pois = encoder_dict["pois"].classes_.shape[0]
n_g2 = encoder_dict["g2"].classes_.shape[0]
n_g3 = encoder_dict["g3"].classes_.shape[0]
n_g4 = encoder_dict["g4"].classes_.shape[0]
n_g5 = encoder_dict["g5"].classes_.shape[0]
n_g6 = encoder_dict["g6"].classes_.shape[0]


# account for the padding token
dims = BaselineDimensions(
    n_users + 1, n_pois + 1, n_g2 + 1, n_g3 + 1, n_g4 + 1, n_g5 + 1, n_g6 + 1
)

In [55]:
wandb.finish()

In [56]:
from lightning.pytorch.loggers import WandbLogger
from lightning.pytorch import Trainer


batch_size = 60
wandb.finish()
torch.cuda.empty_cache()
# cargo-cult like stuff that is supposed to make you faster
torch.set_float32_matmul_precision("medium")
torch.backends.cudnn.benchmark = True

ds = CheckinModule(encoded_data, batch_size=32, workers=4)

wandb.init(project="trovailpoi")

classifier = HMT_RN(dims, embedding_dim=1024, lstm_hidden_dim=1024)
wandb_logger = WandbLogger(project="trovailpoi")
trainer = Trainer(
    max_epochs=200,
    accelerator="auto",
    devices=[0],
    log_every_n_steps=10,
    logger=wandb_logger,
    strategy="auto",
    callbacks=[
        torchpl.callbacks.LearningRateMonitor(logging_interval="step"),
        torchpl.callbacks.ModelCheckpoint(
            monitor="val/loss",
            mode="min",
            save_top_k=1,
            save_last=True,
            filename="best_model",
        ),
        torchpl.callbacks.EarlyStopping(monitor="val/loss", patience=10, mode="min"),
    ],
)

trainer.fit(model=classifier, datamodule=ds)
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: loi-1940849 (poi-dl-airo). Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/dario/foursquare/.venv/lib/python3.12/site-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name           | Type             | Params
-----------------------------------------------------
0  | user_embedding | Embedding        | 20.3 M
1  | poi_embedding  | Embedding        | 4.6 M 
2  | g2_embed       | Embedding        | 114 K 
3  | g3_embed       | Embedding        | 287 K 
4  | g4_embed       | Embedding        | 561 K 
5  | g5_embed       | Embedding        | 1.4 M 
6  | g6_embed       | Embedding        | 2.8 M 
7  | e_drop         | Dropout          | 0     
8  | lstm           | LSTM      

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇██
lr-AdamW,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss,▆█▇▃▅▅▄▂▄▅▁▃▃▃▂▄▃▃▂▃▄▁▃▂▂▄▁▂▃▃▄▂▂▁▁▂▂▂▄▃
train/loss_gap2,█▆▄▂▄▃▃▂▄▃▁▂▂▂▂▄▂▂▃▃▄▁▂▃▂▅▁▂▄▃▄▃▃▂▂▂▂▂▅▂
train/loss_gap3,█▇▆▃▅▄▃▂▄▃▁▃▂▂▂▄▃▃▃▃▅▁▂▃▂▅▁▃▄▃▄▃▃▂▂▂▂▂▄▃
train/loss_gap4,▇██▃▅▄▄▂▄▄▁▃▂▂▂▄▂▃▃▃▅▁▃▃▃▅▁▂▄▃▄▃▃▂▂▂▃▂▅▃
train/loss_gap5,▅█▇▄▅▄▄▂▃▄▁▃▂▂▁▄▂▃▂▃▃▁▃▂▂▄▁▂▃▃▃▂▂▁▁▂▂▂▄▃
train/loss_gap6,▄█▇▄▅▅▄▂▄▅▁▃▃▃▁▄▂▃▂▃▃▁▂▂▂▄▁▂▃▂▃▂▂▁▁▂▂▂▄▃
train/loss_poi,▄█▇▃▄▅▄▂▄▆▁▃▃▄▂▄▃▃▂▃▄▂▃▂▃▄▁▂▃▃▃▃▂▂▁▂▂▂▅▃
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val/loss,█▆▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁


## Scrapbook for Experimentation

Ignore all code below, it's just for quick prototyping